**Introduction**
This kernel was made to formalize reviews for the prescripted medicine and considers how such reviews might develop in the future. Medication feedback made by customers can help them find the most cost-effectiveness drug and for pharmacists there is good evidence that medication review improves process outcomes of prescribing and spot out specific problems early on. 
We will use MAE for this because we don't need positive errors to cancel out negative ones or to give a relatively high weight to large errors.

In [ ]:
import os, pickle
import re, gc
import keras
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

Let’s prepare the data.

Tip: you can use "config style" for code reusability and faster experiments

In [ ]:
conf = {
    "kaggle" : True,
    "embedding_file" : '../input/glove840b300dtxt/glove.840B.300d.txt',
    "embedding_pickle" : '/data/glove.840B.300d/glove.840B.300d.pickle',
    "train_path" : "../input/kuc-hackathon-winter-2018/drugsComTrain_raw.csv",
    "test_path" : "../input/kuc-hackathon-winter-2018/drugsComTest_raw.csv",
    
    
    "max_features" : 30000,
    "maxlen" : 100,
    "embed_size" : 300,
    
    
    "batch_size":3000,
    "epochs":500,
    "n_splits":10,
    "random_state":0
}

model_conf = {
    "SpDr": 0.2,
    "GRU_Units":128,
    "Conv_Units":128,
    "conv_kernel":1,
    "Dense1_Unit":128,
    "Dr1":0.2,
    "Dense2_Unit":128,
    "Dr2": 0.2,
    
    
    "lr" : 1e-4,
    "loss" : 'mean_absolute_error',
    "metrics": ['mae']#list
}

The first thing we must check is the distribution of rating.This graph is bimodal distribution due to the reasons why customers write review in the first place.

In [ ]:
d_train = pd.read_csv(conf["train_path"])
d_test = pd.read_csv(conf["test_path"])

plt.figure(figsize=(8,8))
sns.distplot(d_train['rating'])

plt.xlabel('Rating')
plt.ylabel('Dist')
plt.title("Distribution of rating")
plt.show()

We examine the total amount of rewiews made per year. Growth from year 2015 and later have reasons (maybe social) unimportant for the current goal

In [ ]:
d_train['year'] = pd.to_datetime(d_train['date'], errors='coerce')
cnt = d_train['year'].dt.year.value_counts()
cnt = cnt.sort_index()
plt.figure(figsize=(9,6))
sns.barplot(cnt.index, cnt.values,color='blue',alpha=0.4)
plt.xticks(rotation='vertical')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title("Reviews per year")
plt.show()

We examine the total amount of rewiews made per month. The graph is constant so it isn't really important.

In [ ]:
d_train['month'] = pd.to_datetime(d_train['date'], errors='coerce')
cnt = d_train['month'].dt.month.value_counts()
cnt = cnt.sort_index()
plt.figure(figsize=(9,6))
sns.barplot(cnt.index, cnt.values,color='blue',alpha=0.4)
plt.xticks(rotation='vertical')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title("Reviews per month")
plt.show()

We examine the total amount of rewiews made per day. As the previous one, the graph is constant .

In [ ]:
d_train['day'] = pd.to_datetime(d_train['date'], errors='coerce')
cnt = d_train['day'].dt.day.value_counts()
cnt = cnt.sort_index()
plt.figure(figsize=(9,6))
sns.barplot(cnt.index, cnt.values,color='blue',alpha=0.4)
plt.xticks(rotation='vertical')
plt.xlabel('Day', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title("Reviews per day")
plt.show()

Top 10 popular drugs. Birth control and antidepressants are the most common drugs to see on top

In [ ]:
drug = d_train['drugName'].value_counts()

plt.figure(figsize=(20,6))

sns.barplot(drug[:15].index, drug[:15],color='blue',alpha=0.4)

plt.xlabel('Name of drug', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.title("Top 10 popular drugs")
plt.xticks(rotation=90)
plt.show()

Number of users who found review useful. Gamma distribution it is

In [ ]:
plt.figure(figsize=(16,8))

sns.distplot(d_train['usefulCount'].value_counts())


plt.xlabel('Number of users who found review useful')
plt.ylabel('Dist')
plt.title("Distribution of usefulCount")


plt.show()

Correlation between rating and users who found review useful. Positive linear correlation

Acording to this we decided to use **only** text data for preventing major **data leak** and, you know, be more usefull in real world application

In [ ]:
plt.figure(figsize=(8,6))

sns.lineplot(x='rating',y='usefulCount',data=d_train)

Test the distribution of ratings in train and test dataset. Graphs are similar, the data was choosen correctly 

In [ ]:
plt.figure(figsize=(9,9))

sns.distplot(d_train['rating'])
sns.distplot(d_test['rating'],color='violet')

plt.xlabel('Rating')
plt.ylabel('Dist')
plt.title("Distribution of rating (train and test)")

plt.show()

For easier predictions we divide ratings by 10

In [ ]:
train = pd.read_csv(conf["train_path"])
test = pd.read_csv(conf["test_path"])

X_train = train["review"]
y_train = train["rating"].values / 10

X_test = test["review"]
sample_pred = np.zeros_like(test["rating"], dtype=np.float32)

tokenizer = text.Tokenizer(num_words=conf["max_features"])
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=conf["maxlen"])
x_test = sequence.pad_sequences(X_test, maxlen=conf["maxlen"])

Split reviews by words  and do embedding matrix

We choose Glove embedding, cause we earned best scores here

In [ ]:
if conf["kaggle"]: #tip: cause of config code style you can use same code of local experiments & for kaggle submission
    def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(conf["embedding_file"],encoding="UTF-8"))
else:
    with open(conf["embedding_pickle"], 'rb') as handle: #if you dont want to wait 6-9 minuts you can pickle your dict once
        embeddings_index = pickle.load(handle) # and just unpickle next, it may take near 10-20 seconds
    
word_index = tokenizer.word_index
nb_words = min(conf["max_features"], len(word_index))
embedding_matrix = np.zeros((nb_words, conf["embed_size"]))
for word, i in word_index.items():
    if i >= conf["max_features"]: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
conf["embedding_matrix"] = embedding_matrix

del embeddings_index
gc.collect()

Writing attention mechanism.

Because word processing task can be described as sequence processing task we make two layered lstm with Attention to prevent overfitting and data augmentation

Actually stolen from: https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043

In [ ]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    
    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim


We chose to combine 2 kinds of models: 

First is really complicated 2 layer LSTM model with Attention layer for increasing information "capability", its model focusing on catching sequence information. Also checkout

this: https://arxiv.org/pdf/1709.00893v1.pdf

and this: https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043  we changed alot, but base arcitecture was took

In [ ]:
def model_lstm_atten(main_conf: dict, model_conf:dict):
    inp = Input(shape=(main_conf["maxlen"], ))
    x = Embedding(main_conf["max_features"], main_conf["embed_size"], weights=[main_conf["embedding_matrix"]],trainable = False)(inp)
    x = Bidirectional(CuDNNLSTM(model_conf["LSTM_first_layer_units"], return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(model_conf["LSTM_second_layer_units"], return_sequences=True))(x)
    x = Attention(main_conf["maxlen"])(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(1, activation="relu")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss=model_conf["loss"],optimizer=Adam(lr=model_conf["lr"]),metrics=model_conf["metrics"])
    
    return model

Second is old but still attrective GRU->CNN->Dense, was used by us in Toxic Classification Challange and it gave us bronze medal:)

This model is more about catchaning structure of sentences. Also checkout this:

I havent find original paper(cause it wasnt on arxiv(facepalm)) but you can look here: https://arxiv.org/pdf/1806.11316v1.pdf

In [ ]:
def get_toxic_model(main_conf: dict, model_conf:dict):
    sequence_input = Input(shape=(main_conf["maxlen"], ))
    x = Embedding(main_conf["max_features"], main_conf["embed_size"], weights=[main_conf["embedding_matrix"]],trainable = False)(sequence_input)
    x = SpatialDropout1D(model_conf["SpDr"])(x)
    x = Bidirectional(CuDNNGRU(model_conf["GRU_Units"], return_sequences=True))(x)
    x = Conv1D(model_conf["Conv_Units"], kernel_size = model_conf["conv_kernel"], padding = "valid", kernel_initializer = "glorot_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])
    x = Dense(model_conf["Dense1_Unit"], activation='relu')(x)
    x = Dropout(model_conf["Dr1"])(x)
    x = Dense(model_conf["Dense2_Unit"], activation='relu')(x)
    x = Dropout(model_conf["Dr2"])(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
    model.compile(loss=model_conf["loss"],optimizer=Adam(lr=model_conf["lr"]),metrics=model_conf["metrics"])
    return model

Using out of fold predictions and  blending them

In [ ]:
kf = KFold(n_splits=conf["n_splits"], shuffle=True, random_state=conf["random_state"])
cnt = 0
model_conf["LSTM_first_layer_units"] = 128
model_conf["LSTM_second_layer_units"] = 64

for train_index, test_index in kf.split(x_train, y_train):
    model = model_lstm_atten(conf, model_conf)
    es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')

    hist = model.fit(x_train[train_index], y_train[train_index], batch_size=conf["batch_size"], epochs=conf["epochs"], 
                     validation_data=(x_train[test_index], y_train[test_index]),callbacks=[es])
    model.save(f"model_lstm_atten_cv_{cnt}.keras")
    
    sample_pred +=  model.predict(x_test, conf["batch_size"]*2).reshape(-1)
    cnt+=1

In [ ]:
kf = KFold(n_splits=conf["n_splits"], shuffle=True, random_state=conf["random_state"])
cnt = 0

for train_index, test_index in kf.split(x_train, y_train):
    model = get_toxic_model(conf, model_conf)
    es = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')

    hist = model.fit(x_train[train_index], y_train[train_index], batch_size=conf["batch_size"], epochs=conf["epochs"], 
                     validation_data=(x_train[test_index], y_train[test_index]),callbacks=[es])
    model.save(f"model_gru_cnn_{cnt}.keras")
    
    sample_pred +=  model.predict(x_test, conf["batch_size"]*2).reshape(-1)
    cnt+=1

In [ ]:
print(mean_absolute_error(test["rating"], 10 * sample_pred/ conf["n_splits"]/2))
